<div style="text-align: center; background-color: #b1d1ff; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
Question-based Split
</div>

## Push to Hugging Face

In [1]:
import os
from dotenv import load_dotenv

import json
from collections import defaultdict
import os
import random

# Load .env file
load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")

In [2]:
from huggingface_hub import login

login(token=HF_TOKEN)

list_conversations_train = []
list_conversations_test = []


d:\Tyan\Quickom\Simple-Fine-Tune-Llama\finetune_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
def split_dataset(input_file: str, train_set, test_set):
    # Kiểm tra file có tồn tại không
    if not os.path.exists(input_file):
        print(f"Lỗi: Không tìm thấy file '{input_file}'")
        return

    print(f"Đang đọc dữ liệu từ {input_file}")
    
    # 2. Đọc file JSON
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            data = json.load(f) # Load toàn bộ nội dung file thành list dictionary
    except json.JSONDecodeError:
        print("Lỗi: File JSON bị lỗi format (hoặc không đúng chuẩn JSON array).")
        return

    print(f"\033[92m Tổng số dòng dữ liệu đọc được: {len(data)} \033[0m")

    # 3. Group data theo hotel_name
    grouped_data = defaultdict(list)
    for item in data:
        # Kiểm tra xem item có key 'hotel_name' không để tránh lỗi
        if 'hotel_name' in item:
            grouped_data[item['hotel_name']].append(item)
    
    print(f"Tìm thấy {len(grouped_data)} khách sạn khác nhau.")

    # Đặt seed để mỗi lần chạy code đều chia giống nhau (quan trọng để tái lập kết quả)
    random.seed(42) 

    for hotel, items in grouped_data.items():
        # Xáo trộn thứ tự câu hỏi của khách sạn này
        random.shuffle(items)
        
        total_items = len(items)
        
        # Tính điểm cắt 90%
        split_index = int(total_items * 0.9)
        
        # Logic xử lý cho các trường hợp đặc biệt (ít dữ liệu)
        # Nếu chỉ có 1 dòng data -> ưu tiên đưa vào Train (để model học được cái gì đó)
        if total_items > 0 and split_index == 0:
            split_index = 1
            
        train_subset = items[:split_index]
        test_subset = items[split_index:]

        print(f"'{hotel:<75}' : Tổng {total_items:>3} mục -> Train: {len(train_subset):>3}, Test: {len(test_subset):>3}")
        
        train_set.extend(train_subset)
        test_set.extend(test_subset)

In [4]:
file_path = "./finetuned-datasets/5-hotel_addresses.json"
split_dataset(file_path, list_conversations_train, list_conversations_test)

Đang đọc dữ liệu từ ./finetuned-datasets/5-hotel_addresses.json
 Tổng số dòng dữ liệu đọc được: 1000 
Tìm thấy 100 khách sạn khác nhau.
'Renaissance Riverside Saigon Hotel                                         ' : Tổng  10 mục -> Train:   9, Test:   1
'Ramana Saigon Hotel                                                        ' : Tổng  10 mục -> Train:   9, Test:   1
'D1 Mansion - Zenity Luxury Apartment - Rooftop Pool - Saigon Centre        ' : Tổng  10 mục -> Train:   9, Test:   1
'Saigon Court Serviced Apartment                                            ' : Tổng  10 mục -> Train:   9, Test:   1
'The Hammock Hotel Ben Thanh                                                ' : Tổng  10 mục -> Train:   9, Test:   1
'Mari Queen Hotel                                                           ' : Tổng  10 mục -> Train:   9, Test:   1
'Pullman Saigon Centre Hotel                                                ' : Tổng  10 mục -> Train:   9, Test:   1
'Hotel Iris                           

In [5]:
file_path = "./finetuned-datasets/5-hotel_emails.json"
split_dataset(file_path, list_conversations_train, list_conversations_test)

Đang đọc dữ liệu từ ./finetuned-datasets/5-hotel_emails.json
 Tổng số dòng dữ liệu đọc được: 300 
Tìm thấy 100 khách sạn khác nhau.
'Renaissance Riverside Saigon Hotel                                         ' : Tổng   3 mục -> Train:   2, Test:   1
'Ramana Saigon Hotel                                                        ' : Tổng   3 mục -> Train:   2, Test:   1
'D1 Mansion - Zenity Luxury Apartment - Rooftop Pool - Saigon Centre        ' : Tổng   3 mục -> Train:   2, Test:   1
'Saigon Court Serviced Apartment                                            ' : Tổng   3 mục -> Train:   2, Test:   1
'The Hammock Hotel Ben Thanh                                                ' : Tổng   3 mục -> Train:   2, Test:   1
'Mari Queen Hotel                                                           ' : Tổng   3 mục -> Train:   2, Test:   1
'Pullman Saigon Centre Hotel                                                ' : Tổng   3 mục -> Train:   2, Test:   1
'Hotel Iris                               

In [6]:
file_path = "./finetuned-datasets/5-hotel_phones.json"
split_dataset(file_path, list_conversations_train, list_conversations_test)

Đang đọc dữ liệu từ ./finetuned-datasets/5-hotel_phones.json
 Tổng số dòng dữ liệu đọc được: 300 
Tìm thấy 100 khách sạn khác nhau.
'Renaissance Riverside Saigon Hotel                                         ' : Tổng   3 mục -> Train:   2, Test:   1
'Ramana Saigon Hotel                                                        ' : Tổng   3 mục -> Train:   2, Test:   1
'D1 Mansion - Zenity Luxury Apartment - Rooftop Pool - Saigon Centre        ' : Tổng   3 mục -> Train:   2, Test:   1
'Saigon Court Serviced Apartment                                            ' : Tổng   3 mục -> Train:   2, Test:   1
'The Hammock Hotel Ben Thanh                                                ' : Tổng   3 mục -> Train:   2, Test:   1
'Mari Queen Hotel                                                           ' : Tổng   3 mục -> Train:   2, Test:   1
'Pullman Saigon Centre Hotel                                                ' : Tổng   3 mục -> Train:   2, Test:   1
'Hotel Iris                               

In [7]:
file_path = "./finetuned-datasets/5-hotel_roomtypes.json"
split_dataset(file_path, list_conversations_train, list_conversations_test)

Đang đọc dữ liệu từ ./finetuned-datasets/5-hotel_roomtypes.json
 Tổng số dòng dữ liệu đọc được: 1000 
Tìm thấy 101 khách sạn khác nhau.
'Renaissance Riverside Saigon Hotel                                         ' : Tổng   9 mục -> Train:   8, Test:   1
'Renaissance Riverside Saigon  Hotel                                        ' : Tổng   1 mục -> Train:   1, Test:   0
'Ramana Saigon Hotel                                                        ' : Tổng  10 mục -> Train:   9, Test:   1
'D1 Mansion - Zenity Luxury Apartment - Rooftop Pool - Saigon Centre        ' : Tổng  10 mục -> Train:   9, Test:   1
'Saigon Court Serviced Apartment                                            ' : Tổng  10 mục -> Train:   9, Test:   1
'The Hammock Hotel Ben Thanh                                                ' : Tổng  10 mục -> Train:   9, Test:   1
'Mari Queen Hotel                                                           ' : Tổng  10 mục -> Train:   9, Test:   1
'Pullman Saigon Centre Hotel          

In [8]:
file_path = "./finetuned-datasets/5-hotel_general.json"
split_dataset(file_path, list_conversations_train, list_conversations_test)

Đang đọc dữ liệu từ ./finetuned-datasets/5-hotel_general.json
 Tổng số dòng dữ liệu đọc được: 1000 
Tìm thấy 100 khách sạn khác nhau.
'Renaissance Riverside Saigon Hotel                                         ' : Tổng  10 mục -> Train:   9, Test:   1
'Ramana Saigon Hotel                                                        ' : Tổng  10 mục -> Train:   9, Test:   1
'D1 Mansion - Zenity Luxury Apartment - Rooftop Pool - Saigon Centre        ' : Tổng  10 mục -> Train:   9, Test:   1
'Saigon Court Serviced Apartment                                            ' : Tổng  10 mục -> Train:   9, Test:   1
'The Hammock Hotel Ben Thanh                                                ' : Tổng  10 mục -> Train:   9, Test:   1
'Mari Queen Hotel                                                           ' : Tổng  10 mục -> Train:   9, Test:   1
'Pullman Saigon Centre Hotel                                                ' : Tổng  10 mục -> Train:   9, Test:   1
'Hotel Iris                             

In [10]:
file_path = "./finetuned-datasets/5-baseline_general.json"

if file_path.endswith(".json"):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        print(f"Loaded {file_path} with {len(data)} conversations.")
        if isinstance(data, list):
            list_conversations_train.extend(data)

print(f"Total conversations loaded: {len(list_conversations_train)}")

Loaded ./finetuned-datasets/5-baseline_general.json with 200 conversations.
Total conversations loaded: 3300


In [13]:
len(list_conversations_train), len(list_conversations_test)

(3300, 500)

In [12]:
from huggingface_hub import HfApi

dataset_name = "tyanfarm/hotels-questions-answers-mixed"
api = HfApi()
api.create_repo(repo_id=dataset_name, repo_type="dataset", exist_ok=True)

RepoUrl('https://huggingface.co/datasets/tyanfarm/hotels-questions-answers-mixed', endpoint='https://huggingface.co', repo_type='dataset', repo_id='tyanfarm/hotels-questions-answers-mixed')

In [14]:
from datasets import Dataset, DatasetDict

train_ds = Dataset.from_list(list_conversations_train)
test_ds = Dataset.from_list(list_conversations_test)
dataset = DatasetDict({
    "train": train_ds,
    "test": test_ds,
})

dataset.push_to_hub(dataset_name)

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.76s/ shards]
d:\Tyan\Quickom\Simple-Fine-Tune-Llama\finetune_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Tyan\.cache\huggingface\hub\datasets--tyanfarm--hotels-questions-answers-mixed. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(mess

CommitInfo(commit_url='https://huggingface.co/datasets/tyanfarm/hotels-questions-answers-mixed/commit/e29ef9ea13cf42de60d5e445a6b6ba9c2be71cf7', commit_message='Upload dataset', commit_description='', oid='e29ef9ea13cf42de60d5e445a6b6ba9c2be71cf7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/tyanfarm/hotels-questions-answers-mixed', endpoint='https://huggingface.co', repo_type='dataset', repo_id='tyanfarm/hotels-questions-answers-mixed'), pr_revision=None, pr_num=None)